In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df = pd.read_csv('investments_VC.csv', encoding= 'ISO-8859-1')
df.head()

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,"40,000",operating,EST,NaN,Tallinn,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,"15,00,000",operating,GBR,NaN,London,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df_1=df #备份

# Data Cleaning

In [4]:
df.columns = df.columns.str.replace(' ', '')

In [5]:
# the funding total column is read as a object so clearning it up so that we can use it as a numerical column
df['funding_total_usd']=df['funding_total_usd'].str.replace(',','') # removing commas from funding_total_usd column
df['funding_total_usd']=df['funding_total_usd'].str.replace(' ','')#removing extra space from funding_total_usd column
df['funding_total_usd']=df['funding_total_usd'].str.replace('-','0') #removing - from funding_total_usd column and replacing with 0
df['funding_total_usd'] = pd.to_numeric(df['funding_total_usd'],errors='coerce')# turning column to number

In [6]:
#turning all date columns in to date
df['founded_at'] =  pd.to_datetime(df['founded_at'], format='%Y-%m-%d', errors = 'coerce') # conveting column into date and ignoring errors
df['first_funding_at'] =  pd.to_datetime(df['first_funding_at'], format='%Y-%m-%d', errors = 'coerce')  # conveting column into date and ignoring errors
df['last_funding_at'] =  pd.to_datetime(df['last_funding_at'], format='%Y-%m-%d', errors = 'coerce')  # conveting column into date and ignoring errors
df['founded_year'] =  pd.to_datetime(df['founded_year'], format='%Y', errors = 'coerce') # conveting column into date and ignoring errors
df['founded_month'] =  pd.to_datetime(df['founded_month'], format='%Y-%m', errors = 'coerce') # conveting column into date and ignoring errors

df['market'] = df['market'].str.strip() #removing space from beginnning and end of market column

In [7]:
columns_to_fill_zero = ['seed', 'venture', 'equity_crowdfunding', 'undisclosed', 'convertible_note',
                         'debt_financing', 'angel', 'grant', 'private_equity', 'post_ipo_equity',
                         'post_ipo_debt', 'secondary_market', 'product_crowdfunding', 'round_A',
                         'round_B', 'round_C', 'round_D', 'round_E', 'round_F', 'round_G', 'round_H']

# 使用fillna将上面列的空值填充为0
df[columns_to_fill_zero] = df[columns_to_fill_zero].fillna(0)

# Feature Engineering

In [8]:
from sklearn.preprocessing import LabelEncoder 
import re

> **Country**

In [9]:
# counrtry level comparison
country = pd.read_csv('country.csv') # uploading dataset to map countyr code to continent
country = country[['alpha-3','region','sub-region','region-code','sub-region-code']]
country = country.dropna(how='any',axis=0)
country.isnull().sum() # checking if there is null values
df1 = df.merge(country, left_on='country_code', right_on='alpha-3')
print(df1.head())
df1['sub_region'] = df1['sub-region'].astype('category')
label_encoder = LabelEncoder()
df1['sub_region_cat'] = label_encoder.fit_transform(df1['sub_region'])

category_mapping_region = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(category_mapping_region)

                         permalink                name  \
0            /organization/waywire            #waywire   
1  /organization/tv-communications  &TV Communications   
2   /organization/r-ranch-and-mine  -R- Ranch and Mine   
3       /organization/club-domains       .Club Domains   
4   /organization/004-technologies    004 Technologies   

                        homepage_url  \
0             http://www.waywire.com   
1              http://enjoyandtv.com   
2                                NaN   
3                   http://nic.club/   
4  http://004gmbh.de/en/004-interact   

                                category_list    market  funding_total_usd  \
0  |Entertainment|Politics|Social Media|News|      News          1750000.0   
1                                     |Games|     Games          4000000.0   
2               |Tourism|Entertainment|Games|   Tourism            60000.0   
3                                  |Software|  Software          7000000.0   
4                   

> **Industry**

In [10]:
# grouping markets in industries to decrease the number of segments. The list was being taken from here https://support.crunchbase.com/hc/en-us/articles/360043146954-What-Industries-are-included-in-Crunchbase-
admin_services = str('Employer Benefits Programs, Human Resource Automation, Corporate IT, Distribution, Service Providers, Archiving Service, Call Center, Collection Agency, College Recruiting, Courier Service, Debt Collections, Delivery, Document Preparation, Employee Benefits, Extermination Service, Facilities Support Services, Housekeeping Service, Human Resources, Knowledge Management, Office Administration, Packaging Services, Physical Security, Project Management, Staffing Agency, Trade Shows, Virtual Workforce').split(', ')
advertising = str('Creative Industries, Promotional, Advertising Ad Exchange, Ad Network, Ad Retargeting, Ad Server, Ad Targeting, Advertising, Advertising Platforms, Affiliate Marketing, Local Advertising, Mobile Advertising, Outdoor Advertising, SEM, Social Media Advertising, Video Advertising').split(', ')
agriculture = str('Agriculture, AgTech, Animal Feed, Aquaculture, Equestrian, Farming, Forestry, Horticulture, Hydroponics, Livestock').split(', ')
app = str('Application Performance Monitoring, App Stores, Application Platforms, Enterprise Application, App Discovery, Apps, Consumer Applications, Enterprise Applications, Mobile Apps, Reading Apps, Web Apps').split(', ')
artificial_intelli = str('Artificial Intelligence, Intelligent Systems, Machine Learning, Natural Language Processing, Predictive Analytics').split(', ')
biotechnology = str('Synthetic Biology, Bio-Pharm, Bioinformatics, Biometrics, Biopharma, Biotechnology, Genetics, Life Science, Neuroscience, Quantified Self').split(', ')
clothing = str('Fashion, Laundry and Dry-cleaning, Lingerie, Shoes').split(', ')
shopping = str('Consumer Behavior, Customer Support Tools, Discounts, Reviews and Recommendations, Auctions, Classifieds, Collectibles, Consumer Reviews, Coupons, E-Commerce, E-Commerce Platforms, Flash Sale, Gift, Gift Card, Gift Exchange, Gift Registry, Group Buying, Local Shopping, Made to Order, Marketplace, Online Auctions, Personalization, Point of Sale, Price Comparison, Rental, Retail, Retail Technology, Shopping, Shopping Mall, Social Shopping, Sporting Goods, Vending and Concessions, Virtual Goods, Wholesale').split(', ')
community = str("Self Development, Sex, Forums, Match-Making, Babies, Identity, Women, Kids, Entrepreneur, Networking, Adult, Baby, Cannabis, Children, Communities, Dating, Elderly, Family, Funerals, Humanitarian, Leisure, LGBT, Lifestyle, Men's, Online Forums, Parenting, Pet, Private Social Networking, Professional Networking, Q&A, Religion, Retirement, Sex Industry, Sex Tech, Social, Social Entrepreneurship, Teenagers, Virtual World, Wedding, Women's, Young Adults").split(', ')
electronics  = str('Mac, iPod Touch, Tablets, iPad, iPhone, Computer, Consumer Electronics, Drones, Electronics, Google Glass, Mobile Devices, Nintendo, Playstation, Roku, Smart Home, Wearables, Windows Phone, Xbox').split(', ')
consumer_goods= str('Commodities, Sunglasses, Groceries, Batteries, Cars, Beauty, Comics, Consumer Goods, Cosmetics, DIY, Drones, Eyewear, Fast-Moving Consumer Goods, Flowers, Furniture, Green Consumer Goods, Handmade, Jewelry, Lingerie, Shoes, Tobacco, Toys').split(', ')
content = str('E-Books, MicroBlogging, Opinions, Blogging Platforms, Content Delivery Network, Content Discovery, Content Syndication, Creative Agency, DRM, EBooks, Journalism, News, Photo Editing, Photo Sharing, Photography, Printing, Publishing, Social Bookmarking, Video Editing, Video Streaming').split(', ')
data = str('Optimization, A/B Testing, Analytics, Application Performance Management, Artificial Intelligence, Big Data, Bioinformatics, Biometrics, Business Intelligence, Consumer Research, Data Integration, Data Mining, Data Visualization, Database, Facial Recognition, Geospatial, Image Recognition, Intelligent Systems, Location Based Services, Machine Learning, Market Research, Natural Language Processing, Predictive Analytics, Product Research, Quantified Self, Speech Recognition, Test and Measurement, Text Analytics, Usability Testing').split(', ')
design = str('Visualization, Graphics, Design, Designers, CAD, Consumer Research, Data Visualization, Fashion, Graphic Design, Human Computer Interaction, Industrial Design, Interior Design, Market Research, Mechanical Design, Product Design, Product Research, Usability Testing, UX Design, Web Design').split(', ')
education = str('Universities, College Campuses, University Students, High Schools, All Students, Colleges, Alumni, Charter Schools, College Recruiting, Continuing Education, Corporate Training, E-Learning, EdTech, Education, Edutainment, Higher Education, Language Learning, MOOC, Music Education, Personal Development, Primary Education, Secondary Education, Skill Assessment, STEM Education, Textbook, Training, Tutoring, Vocational Education').split(', ')
energy = str('Gas, Natural Gas Uses, Oil, Oil & Gas, Battery, Biofuel, Biomass Energy, Clean Energy, Electrical Distribution, Energy, Energy Efficiency, Energy Management, Energy Storage, Fossil Fuels, Fuel, Fuel Cell, Oil and Gas, Power Grid, Renewable Energy, Solar, Wind Energy').split(', ')
events = str('Concerts, Event Management, Event Promotion, Events, Nightclubs, Nightlife, Reservations, Ticketing, Wedding').split(', ')
financial = str('Debt Collecting, P2P Money Transfer, Investment Management, Trading, Accounting, Angel Investment, Asset Management, Auto Insurance, Banking, Bitcoin, Commercial Insurance, Commercial Lending, Consumer Lending, Credit, Credit Bureau, Credit Cards, Crowdfunding, Cryptocurrency, Debit Cards, Debt Collections, Finance, Financial Exchanges, Financial Services, FinTech, Fraud Detection, Funding Platform, Gift Card, Health Insurance, Hedge Funds, Impact Investing, Incubators, Insurance, InsurTech, Leasing, Lending, Life Insurance, Micro Lending, Mobile Payments, Payments, Personal Finance, Prediction Markets, Property Insurance, Real Estate Investment, Stock Exchanges, Trading Platform, Transaction Processing, Venture Capital, Virtual Currency, Wealth Management').split(', ')
food = str('Specialty Foods, Bakery, Brewing, Cannabis, Catering, Coffee, Confectionery, Cooking, Craft Beer, Dietary Supplements, Distillery, Farmers Market, Food and Beverage, Food Delivery, Food Processing, Food Trucks, Fruit, Grocery, Nutrition, Organic Food, Recipes, Restaurants, Seafood, Snack Food, Tea, Tobacco, Wine And Spirits, Winery').split(', ')
gaming = str('Game, Games, Casual Games, Console Games, Contests, Fantasy Sports, Gambling, Gamification, Gaming, MMO Games, Online Games, PC Games, Serious Games, Video Games').split(', ')
government = str('Polling, Governance, CivicTech, Government, GovTech, Law Enforcement, Military, National Security, Politics, Public Safety, Social Assistance').split(', ')
hardware= str('Cable, 3D, 3D Technology, Application Specific Integrated Circuit (ASIC), Augmented Reality, Cloud Infrastructure, Communication Hardware, Communications Infrastructure, Computer, Computer Vision, Consumer Electronics, Data Center, Data Center Automation, Data Storage, Drone Management, Drones, DSP, Electronic Design Automation (EDA), Electronics, Embedded Systems, Field-Programmable Gate Array (FPGA), Flash Storage, Google Glass, GPS, GPU, Hardware, Industrial Design, Laser, Lighting, Mechanical Design, Mobile Devices, Network Hardware, NFC, Nintendo, Optical Communication, Playstation, Private Cloud, Retail Technology, RFID, RISC, Robotics, Roku, Satellite Communication, Semiconductor, Sensor, Sex Tech, Telecommunications, Video Conferencing, Virtual Reality, Virtualization, Wearables, Windows Phone, Wireless, Xbox').split(', ')
health_care = str('Senior Health, Physicians, Electronic Health Records, Doctors, Healthcare Services, Diagnostics, Alternative Medicine, Assisted Living, Assistive Technology, Biopharma, Cannabis, Child Care, Clinical Trials, Cosmetic Surgery, Dental, Diabetes, Dietary Supplements, Elder Care, Electronic Health Record (EHR), Emergency Medicine, Employee Benefits, Fertility, First Aid, Funerals, Genetics, Health Care, Health Diagnostics, Home Health Care, Hospital, Medical, Medical Device, mHealth, Nursing and Residential Care, Nutraceutical, Nutrition, Outpatient Care, Personal Health, Pharmaceutical, Psychology, Rehabilitation, Therapeutics, Veterinary, Wellness').split(', ')
it = str('Distributors, Algorithms, ICT, M2M, Technology, Business Information Systems, CivicTech, Cloud Data Services, Cloud Management, Cloud Security, CMS, Contact Management, CRM, Cyber Security, Data Center, Data Center Automation, Data Integration, Data Mining, Data Visualization, Document Management, E-Signature, Email, GovTech, Identity Management, Information and Communications Technology (ICT), Information Services, Information Technology, Intrusion Detection, IT Infrastructure, IT Management, Management Information Systems, Messaging, Military, Network Security, Penetration Testing, Private Cloud, Reputation, Sales Automation, Scheduling, Social CRM, Spam Filtering, Technical Support, Unified Communications, Video Chat, Video Conferencing, Virtualization, VoIP').split(', ')
internet = str('Online Identity, Cyber, Portals, Web Presence Management, Domains, Tracking, Web Tools, Curated Web, Search, Cloud Computing, Cloud Data Services, Cloud Infrastructure, Cloud Management, Cloud Storage, Darknet, Domain Registrar, E-Commerce Platforms, Ediscovery, Email, Internet, Internet of Things, ISP, Location Based Services, Messaging, Music Streaming, Online Forums, Online Portals, Private Cloud, Product Search, Search Engine, SEM, Semantic Search, Semantic Web, SEO, SMS, Social Media, Social Media Management, Social Network, Unified Communications, Vertical Search, Video Chat, Video Conferencing, Visual Search, VoIP, Web Browsers, Web Hosting').split(', ')
invest = str('Angel Investment, Banking, Commercial Lending, Consumer Lending, Credit, Credit Cards, Financial Exchanges, Funding Platform, Hedge Funds, Impact Investing, Incubators, Micro Lending, Stock Exchanges, Trading Platform, Venture Capital').split(', ')
manufacturing = str('Innovation Engineering, Civil Engineers, Heavy Industry, Engineering Firms, Systems, 3D Printing, Advanced Materials, Foundries, Industrial, Industrial Automation, Industrial Engineering, Industrial Manufacturing, Machinery Manufacturing, Manufacturing, Paper Manufacturing, Plastics and Rubber Manufacturing, Textiles, Wood Processing').split(', ')
media = str('Writers, Creative, Television, Entertainment, Media, Advice, Animation, Art, Audio, Audiobooks, Blogging Platforms, Broadcasting, Celebrity, Concerts, Content, Content Creators, Content Discovery, Content Syndication, Creative Agency, Digital Entertainment, Digital Media, DRM, EBooks, Edutainment, Event Management, Event Promotion, Events, Film, Film Distribution, Film Production, Guides, In-Flight Entertainment, Independent Music, Internet Radio, Journalism, Media and Entertainment, Motion Capture, Music, Music Education, Music Label, Music Streaming, Music Venues, Musical Instruments, News, Nightclubs, Nightlife, Performing Arts, Photo Editing, Photo Sharing, Photography, Podcast, Printing, Publishing, Reservations, Social Media, Social News, Theatre, Ticketing, TV, TV Production, Video, Video Editing, Video on Demand, Video Streaming, Virtual World').split(', ')
message = str('Unifed Communications, Chat, Email, Meeting Software, Messaging, SMS, Unified Communications, Video Chat, Video Conferencing, VoIP, Wired Telecommunications').split(', ')
mobile = str('Android, Google Glass, iOS, mHealth, Mobile, Mobile Apps, Mobile Devices, Mobile Payments, Windows Phone, Wireless').split(', ')
music = str('Audio, Audiobooks, Independent Music, Internet Radio, Music, Music Education, Music Label, Music Streaming, Musical Instruments, Podcast').split(', ')
resource = str('Biofuel, Biomass Energy, Fossil Fuels, Mineral, Mining, Mining Technology, Natural Resources, Oil and Gas, Precious Metals, Solar, Timber, Water, Wind Energy').split(', ')
navigation = str('Maps, Geospatial, GPS, Indoor Positioning, Location Based Services, Mapping Services, Navigation').split(', ')
other = str('Mass Customization, Monetization, Testing, Subscription Businesses, Mobility, Incentives, Peer-to-Peer, Nonprofits, Alumni, Association, B2B, B2C, Blockchain, Charity, Collaboration, Collaborative Consumption, Commercial, Consumer, Crowdsourcing, Customer Service, Desktop Apps, Emerging Markets, Enterprise, Ethereum, Franchise, Freemium, Generation Y, Generation Z, Homeless Shelter, Infrastructure, Knowledge Management, LGBT Millennials, Non Profit, Peer to Peer, Professional Services, Project Management, Real Time, Retirement, Service Industry, Sharing Economy, Small and Medium Businesses, Social Bookmarking, Social Impact, Subscription Service, Technical Support, Underserved Children, Universities').split(', ')
payment = str('Billing, Bitcoin, Credit Cards, Cryptocurrency, Debit Cards, Fraud Detection, Mobile Payments, Payments, Transaction Processing, Virtual Currency').split(', ')
platforms = str('Development Platforms, Android, Facebook, Google, Google Glass, iOS, Linux, macOS, Nintendo, Operating Systems, Playstation, Roku, Tizen, Twitter, WebOS, Windows, Windows Phone, Xbox').split(', ')
privacy = str('Digital Rights Management, Personal Data, Cloud Security, Corrections Facilities, Cyber Security, DRM, E-Signature, Fraud Detection, Homeland Security, Identity Management, Intrusion Detection, Law Enforcement, Network Security, Penetration Testing, Physical Security, Privacy, Security').split(', ')
services = str('Funeral Industry, English-Speaking, Spas, Plumbers, Service Industries, Staffing Firms, Translation, Career Management, Business Services, Services, Accounting, Business Development, Career Planning, Compliance, Consulting, Customer Service, Employment, Environmental Consulting, Field Support, Freelance, Intellectual Property, Innovation Management, Legal, Legal Tech, Management Consulting, Outsourcing, Professional Networking, Quality Assurance, Recruiting, Risk Management, Social Recruiting, Translation Service').split(', ')
realestate= str('Office Space, Self Storage, Brokers, Storage, Home Owners, Self Storage , Realtors, Home & Garden, Utilities, Home Automation, Architecture, Building Maintenance, Building Material, Commercial Real Estate, Construction, Coworking, Facility Management, Fast-Moving Consumer Goods, Green Building, Home and Garden, Home Decor, Home Improvement, Home Renovation, Home Services, Interior Design, Janitorial Service, Landscaping, Property Development, Property Management, Real Estate, Real Estate Investment, Rental Property, Residential, Self-Storage, Smart Building, Smart Cities, Smart Home, Timeshare, Vacation Rental').split(', ')
sales = str('Advertising, Affiliate Marketing, App Discovery, App Marketing, Brand Marketing, Cause Marketing, Content Marketing, CRM, Digital Marketing, Digital Signage, Direct Marketing, Direct Sales, Email Marketing, Lead Generation, Lead Management, Local, Local Advertising, Local Business, Loyalty Programs, Marketing, Marketing Automation, Mobile Advertising, Multi-level Marketing, Outdoor Advertising, Personal Branding, Public Relations, Sales, Sales Automation, SEM, SEO, Social CRM, Social Media Advertising, Social Media Management, Social Media Marketing, Sponsorship, Video Advertising').split(', ')
science = str('Face Recognition, New Technologies, Advanced Materials, Aerospace, Artificial Intelligence, Bioinformatics, Biometrics, Biopharma, Biotechnology, Chemical, Chemical Engineering, Civil Engineering, Embedded Systems, Environmental Engineering, Human Computer Interaction, Industrial Automation, Industrial Engineering, Intelligent Systems, Laser, Life Science, Marine Technology, Mechanical Engineering, Nanotechnology, Neuroscience, Nuclear, Quantum Computing, Robotics, Semiconductor, Software Engineering, STEM Education').split(', ')
software = str('Business Productivity, 3D Technology, Android, App Discovery, Application Performance Management, Apps, Artificial Intelligence, Augmented Reality, Billing, Bitcoin, Browser Extensions, CAD, Cloud Computing, Cloud Management, CMS, Computer Vision, Consumer Applications, Consumer Software, Contact Management, CRM, Cryptocurrency, Data Center Automation, Data Integration, Data Storage, Data Visualization, Database, Developer APIs, Developer Platform, Developer Tools, Document Management, Drone Management, E-Learning, EdTech, Electronic Design Automation (EDA), Embedded Software, Embedded Systems, Enterprise Applications, Enterprise Resource Planning (ERP), Enterprise Software, Facial Recognition, File Sharing, IaaS, Image Recognition, iOS, Linux, Machine Learning, macOS, Marketing Automation, Meeting Software, Mobile Apps, Mobile Payments, MOOC, Natural Language Processing, Open Source, Operating Systems, PaaS, Predictive Analytics, Presentation Software, Presentations, Private Cloud, Productivity Tools, QR Codes, Reading Apps, Retail Technology, Robotics, SaaS, Sales Automation, Scheduling, Sex Tech, Simulation, SNS, Social CRM, Software, Software Engineering, Speech Recognition, Task Management, Text Analytics, Transaction Processing, Video Conferencing, Virtual Assistant, Virtual Currency, Virtual Desktop, Virtual Goods, Virtual Reality, Virtual World, Virtualization, Web Apps, Web Browsers, Web Development').split(', ')
sports = str('American Football, Baseball, Basketball, Boating, Cricket, Cycling, Diving, eSports, Fantasy Sports, Fitness, Golf, Hockey, Hunting, Outdoors, Racing, Recreation, Rugby, Sailing, Skiing, Soccer, Sporting Goods, Sports, Surfing, Swimming, Table Tennis, Tennis, Ultimate Frisbee, Volley Ball').split(', ')
sustainability = str('Green, Wind, Biomass Power Generation, Renewable Tech, Environmental Innovation, Renewable Energies, Clean Technology, Biofuel, Biomass Energy, Clean Energy, CleanTech, Energy Efficiency, Environmental Engineering, Green Building, Green Consumer Goods, GreenTech, Natural Resources, Organic, Pollution Control, Recycling, Renewable Energy, Solar, Sustainability, Waste Management, Water Purification, Wind Energy').split(', ')
transportation = str('Taxis, Air Transportation, Automotive, Autonomous Vehicles, Car Sharing, Courier Service, Delivery Service, Electric Vehicle, Ferry Service, Fleet Management, Food Delivery, Freight Service, Last Mile Transportation, Limousine Service, Logistics, Marine Transportation, Parking, Ports and Harbors, Procurement, Public Transportation, Railroad, Recreational Vehicles, Ride Sharing, Same Day Delivery, Shipping, Shipping Broker, Space Travel, Supply Chain Management, Taxi Service, Transportation, Warehousing, Water Transportation').split(', ')
travel = str('Adventure Travel, Amusement Park and Arcade, Business Travel, Casino, Hospitality, Hotel, Museums and Historical Sites, Parks, Resorts, Timeshare, Tour Operator, Tourism, Travel, Travel Accommodations, Travel Agency, Vacation Rental').split(', ')
video = str('Animation, Broadcasting, Film, Film Distribution, Film Production, Motion Capture, TV, TV Production, Video, Video Editing, Video on Demand, Video Streaming').split(', ')

#Making new column called  Industry group
df1['Industry_Group'] = np.where(df1.market.str.contains('|'.join(admin_services), flags=re.IGNORECASE), "Administrative Services",
                               np.where(df1.market.str.contains('|'.join(software), flags=re.IGNORECASE), "Software",
                               np.where(df1.market.str.contains('|'.join(advertising), flags=re.IGNORECASE), "Advertising",
                               np.where(df1.market.str.contains('|'.join(agriculture), flags=re.IGNORECASE), "Agriculture and Farming",
                               np.where(df1.market.str.contains('|'.join(app), flags=re.IGNORECASE), "Apps",
                               np.where(df1.market.str.contains('|'.join(artificial_intelli), flags=re.IGNORECASE), "Artificial Intelligence",
                               np.where(df1.market.str.contains('|'.join(biotechnology), flags=re.IGNORECASE), "Biotechnology",
                               np.where(df1.market.str.contains('|'.join(clothing), flags=re.IGNORECASE), "Clothing and Apparel",
                               np.where(df1.market.str.contains('|'.join(shopping), flags=re.IGNORECASE), "Commerce and Shopping",
                               np.where(df1.market.str.contains('|'.join(community), flags=re.IGNORECASE), "Community and Lifestyle",
                               np.where(df1.market.str.contains('|'.join(electronics), flags=re.IGNORECASE), "Consumer Electronics",
                               np.where(df1.market.str.contains('|'.join(consumer_goods), flags=re.IGNORECASE), "Consumer Goods",
                               np.where(df1.market.str.contains('|'.join(content), flags=re.IGNORECASE), "Content and Publishing",
                               np.where(df1.market.str.contains('|'.join(data), flags=re.IGNORECASE), "Data and Analytics",
                               np.where(df1.market.str.contains('|'.join(design), flags=re.IGNORECASE), "Design",
                               np.where(df1.market.str.contains('|'.join(education), flags=re.IGNORECASE), "Education",
                               np.where(df1.market.str.contains('|'.join(energy), flags=re.IGNORECASE), "Energy",
                               np.where(df1.market.str.contains('|'.join(events), flags=re.IGNORECASE), "Events",
                               np.where(df1.market.str.contains('|'.join(financial), flags=re.IGNORECASE), "Financial Services",
                               np.where(df1.market.str.contains('|'.join(food), flags=re.IGNORECASE), "Food and Beverage",
                               np.where(df1.market.str.contains('|'.join(gaming), flags=re.IGNORECASE), "Gaming",
                               np.where(df1.market.str.contains('|'.join(government), flags=re.IGNORECASE), "Government and Military",
                               np.where(df1.market.str.contains('|'.join(hardware), flags=re.IGNORECASE), "Hardware",
                               np.where(df1.market.str.contains('|'.join(health_care), flags=re.IGNORECASE), "Health Care",
                               np.where(df1.market.str.contains('|'.join(it), flags=re.IGNORECASE), "Information Technology",
                               np.where(df1.market.str.contains('|'.join(internet), flags=re.IGNORECASE), "Internet Services",
                               np.where(df1.market.str.contains('|'.join(invest), flags=re.IGNORECASE), "Lending and Investments",
                               np.where(df1.market.str.contains('|'.join(manufacturing), flags=re.IGNORECASE), "Manufacturing",
                               np.where(df1.market.str.contains('|'.join(media), flags=re.IGNORECASE), "Media and Entertainment",
                               np.where(df1.market.str.contains('|'.join(message), flags=re.IGNORECASE), "Messaging and Telecommunication",
                               np.where(df1.market.str.contains('|'.join(mobile), flags=re.IGNORECASE), "Mobile",
                               np.where(df1.market.str.contains('|'.join(music), flags=re.IGNORECASE), "Music and Audio",
                               np.where(df1.market.str.contains('|'.join(resource), flags=re.IGNORECASE), "Natural Resources",
                               np.where(df1.market.str.contains('|'.join(navigation), flags=re.IGNORECASE), "Navigation and Mapping",
                               np.where(df1.market.str.contains('|'.join(payment), flags=re.IGNORECASE), "Payments",
                               np.where(df1.market.str.contains('|'.join(platforms), flags=re.IGNORECASE), "Platforms",
                               np.where(df1.market.str.contains('|'.join(privacy), flags=re.IGNORECASE), "Privacy and Security",
                               np.where(df1.market.str.contains('|'.join(services), flags=re.IGNORECASE), "Professional Services",
                               np.where(df1.market.str.contains('|'.join(realestate), flags=re.IGNORECASE), "Real Estate",
                               np.where(df1.market.str.contains('|'.join(sales), flags=re.IGNORECASE), "Sales and Marketing",
                               np.where(df1.market.str.contains('|'.join(science), flags=re.IGNORECASE), "Science and Engineering",
                               np.where(df1.market.str.contains('|'.join(sports), flags=re.IGNORECASE), "Sports",
                               np.where(df1.market.str.contains('|'.join(sustainability), flags=re.IGNORECASE), "Sustainability",
                               np.where(df1.market.str.contains('|'.join(transportation), flags=re.IGNORECASE), "Transportation",
                               np.where(df1.market.str.contains('|'.join(travel), flags=re.IGNORECASE), "Travel and Tourism",
                               np.where(df1.market.str.contains('|'.join(video), flags=re.IGNORECASE), "Video",
                               np.where(df1.market.str.contains('|'.join(other), flags=re.IGNORECASE), "Other",  "Other")))))))))))))))))))))))))))))))))))))))))))))))

C:\Users\12048\AppData\Local\Temp\ipykernel_26496\1760050934.py:52: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  np.where(df1.market.str.contains('|'.join(software), flags=re.IGNORECASE), "Software",
C:\Users\12048\AppData\Local\Temp\ipykernel_26496\1760050934.py:73: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  np.where(df1.market.str.contains('|'.join(hardware), flags=re.IGNORECASE), "Hardware",
C:\Users\12048\AppData\Local\Temp\ipykernel_26496\1760050934.py:74: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  np.where(df1.market.str.contains('|'.join(health_care), flags=re.IGNORECASE), "Health Care",
C:\Users\12048\AppData\Local\Temp\ipykernel_26496\1760050934.py:75: UserWarning: This pattern is interpreted as a regular ex

In [11]:
df1['Industry_Group'] = df1['Industry_Group'].astype('category')
label_encoder = LabelEncoder()
df1['cat_Industry_Group'] = label_encoder.fit_transform(df1['Industry_Group'])
df1.head()


,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region_x,...,round_H,alpha-3,region_y,sub-region,region-code,sub-region-code,sub_region,sub_region_cat,Industry_Group,cat_Industry_Group
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,1750000.0,acquired,USA,NY,New York City,...,0.0,USA,Americas,Northern America,19.0,21.0,Northern America,6,Content and Publishing,11
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,4000000.0,operating,USA,CA,Los Angeles,...,0.0,USA,Americas,Northern America,19.0,21.0,Northern America,6,Gaming,19
2,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,60000.0,operating,USA,TX,Dallas,...,0.0,USA,Americas,Northern America,19.0,21.0,Northern America,6,Travel and Tourism,42
3,/organization/club-domains,.Club Domains,http://nic.club/,|Software|,Software,7000000.0,NaN,USA,FL,Ft. Lauderdale,...,0.0,USA,Americas,Northern America,19.0,21.0,Northern America,6,Software,38
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,|Software|,Software,0.0,operating,USA,IL,"Springfield, Illinois",...,0.0,USA,Americas,Northern America,19.0,21.0,Northern America,6,Software,38


In [12]:
category_mapping_industry = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
# Display the mapping
print(category_mapping_industry)

{'Administrative Services': 0, 'Advertising': 1, 'Agriculture and Farming': 2, 'Apps': 3, 'Artificial Intelligence': 4, 'Biotechnology': 5, 'Clothing and Apparel': 6, 'Commerce and Shopping': 7, 'Community and Lifestyle': 8, 'Consumer Electronics': 9, 'Consumer Goods': 10, 'Content and Publishing': 11, 'Data and Analytics': 12, 'Design': 13, 'Education': 14, 'Energy': 15, 'Events': 16, 'Financial Services': 17, 'Food and Beverage': 18, 'Gaming': 19, 'Government and Military': 20, 'Hardware': 21, 'Health Care': 22, 'Information Technology': 23, 'Internet Services': 24, 'Manufacturing': 25, 'Media and Entertainment': 26, 'Messaging and Telecommunication': 27, 'Mobile': 28, 'Natural Resources': 29, 'Navigation and Mapping': 30, 'Other': 31, 'Platforms': 32, 'Privacy and Security': 33, 'Professional Services': 34, 'Real Estate': 35, 'Sales and Marketing': 36, 'Science and Engineering': 37, 'Software': 38, 'Sports': 39, 'Sustainability': 40, 'Transportation': 41, 'Travel and Tourism': 42}


> **Investment bounds**

In [13]:
# 获取投资每一列不为0的数目
non_zero_counts = df1.loc[:, 'seed':'product_crowdfunding'].astype(bool).sum()

print(non_zero_counts)  

seed                    11957
venture                 21935
equity_crowdfunding       493
undisclosed               791
convertible_note          520
debt_financing           4104
angel                    2755
grant                    1027
private_equity           1295
post_ipo_equity           305
post_ipo_debt              74
secondary_market           18
product_crowdfunding      183
dtype: int64


In [14]:
# seed degree
df1.loc[(df1["seed"]>= 2000000),"seed_degree"] = "High_seed"
df1.loc[(df1["seed"]< 2000000)& (df1["seed"]>= 150000),"seed_degree"] = "Middle_seed"
df1.loc[(df1["seed"]< 150000)& (df1["seed"]>= 50000),"seed_degree"] = "MiddleLow_seed"
df1.loc[(df1["seed"]< 50000)& (df1["seed"]>0),"seed_degree"] = "Low_seed"
df1.loc[(df1["seed"]==0),"seed_degree"] = "No_seed"
df1['seed_degree'].value_counts()

No_seed           32186
Middle_seed        6468
Low_seed           2107
MiddleLow_seed     1906
High_seed          1476
Name: seed_degree, dtype: int64

In [15]:
# Venture degree
df1.loc[(df1["venture"]>= 100000000),"venture_degree"] = "High_venture"
df1.loc[(df1["venture"]< 100000000)& (df1["venture"]>= 8000000),"venture_degree"] = "Middle_venture"
df1.loc[(df1["venture"]< 8000000)& (df1["venture"]>= 1000000),"venture_degree"] = "MiddleLow_venture"
df1.loc[(df1["venture"]< 1000000)& (df1["venture"]>0),"venture_degree"] = "Low_venture"
df1.loc[(df1["venture"]==0),"venture_degree"] = "No_venture"
df1['venture_degree'].value_counts()

No_venture           22208
MiddleLow_venture     9128
Middle_venture        8737
Low_venture           3581
High_venture           489
Name: venture_degree, dtype: int64

In [16]:
# Angel degree
df1.loc[(df1["angel"]>= 1000000),"angel_degree"] = "High_angel"
df1.loc[(df1["angel"]< 1000000)& (df1["angel"]>= 500000),"angel_degree"] = "Middle_angel"
df1.loc[(df1["angel"]< 500000)& (df1["angel"]>= 100000),"angel_degree"] = "MiddleLow_angel"
df1.loc[(df1["angel"]< 100000)& (df1["angel"]>0),"angel_degree"] = "Low_angel"
df1.loc[(df1["angel"]==0),"angel_degree"] = "No_angel"
df1['angel_degree'].value_counts()

No_angel           41388
MiddleLow_angel      999
High_angel           879
Middle_angel         608
Low_angel            269
Name: angel_degree, dtype: int64

In [17]:
# debt_financing degree
df1.loc[(df1["debt_financing"]>= 2000000),"debt_financing_degree"] = "High_debt_financing"
df1.loc[(df1["debt_financing"]< 2000000)& (df1["debt_financing"]>= 1500000),"debt_financing_degree"] = "Middle_debt_financing"
df1.loc[(df1["debt_financing"]< 1500000)& (df1["debt_financing"]>= 500000),"debt_financing_degree"] = "MiddleLow_debt_financing"
df1.loc[(df1["debt_financing"]< 500000)& (df1["debt_financing"]>0),"debt_financing_degree"] = "Low_debt_financing"
df1.loc[(df1["debt_financing"]==0),"debt_financing_degree"] = "No_debt_financing"
df1['debt_financing_degree'].value_counts()

No_debt_financing           40039
High_debt_financing          1788
Low_debt_financing           1168
MiddleLow_debt_financing      906
Middle_debt_financing         242
Name: debt_financing_degree, dtype: int64

In [18]:
# Other investment

df1['Other_Investment'] = df1['equity_crowdfunding'] +df1['undisclosed']+df1['convertible_note']+df1['private_equity'] +df1['post_ipo_equity'] +df1['post_ipo_debt'] +df1['secondary_market']+df1['product_crowdfunding']
# Other_Investment degree
df1.loc[(df1["Other_Investment"]>= 5000000),"Other_Investment_degree"] = "High_Other_Investment"
df1.loc[(df1["Other_Investment"]< 5000000)& (df1["Other_Investment"]>= 1500000),"Other_Investment_degree"] = "Middle_Other_Investment"
df1.loc[(df1["Other_Investment"]< 1500000)& (df1["Other_Investment"]>= 500000),"Other_Investment_degree"] = "MiddleLow_Other_Investment"
df1.loc[(df1["Other_Investment"]< 500000)& (df1["Other_Investment"]>0),"Other_Investment_degree"] = "Low_Other_Investment"
df1.loc[(df1["Other_Investment"]==0),"Other_Investment_degree"] = "No_Other_Investment"
df1['Other_Investment_degree'].value_counts()

No_Other_Investment           40562
High_Other_Investment          1493
Low_Other_Investment           1081
Middle_Other_Investment         538
MiddleLow_Other_Investment      469
Name: Other_Investment_degree, dtype: int64

In [19]:
df.head()

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,1750000.0,acquired,USA,NY,New York City,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,4000000.0,operating,USA,CA,Los Angeles,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,40000.0,operating,EST,NaN,Tallinn,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,1500000.0,operating,GBR,NaN,London,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,60000.0,operating,USA,TX,Dallas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# 需要编码的列
columns_to_encode = ['seed_degree', 'venture_degree', 'angel_degree', 'debt_financing_degree', 'Other_Investment_degree']

# 创建 LabelEncoder，并创建每个类别到整数的映射
category_mapping = {}

for column in columns_to_encode:
    label_encoder = LabelEncoder()
    df1[column + '_encoded'] = label_encoder.fit_transform(df1[column])
    category_mapping[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# 打印每个类别到整数的映射
print(category_mapping)

{'seed_degree': {'High_seed': 0, 'Low_seed': 1, 'MiddleLow_seed': 2, 'Middle_seed': 3, 'No_seed': 4}, 'venture_degree': {'High_venture': 0, 'Low_venture': 1, 'MiddleLow_venture': 2, 'Middle_venture': 3, 'No_venture': 4}, 'angel_degree': {'High_angel': 0, 'Low_angel': 1, 'MiddleLow_angel': 2, 'Middle_angel': 3, 'No_angel': 4}, 'debt_financing_degree': {'High_debt_financing': 0, 'Low_debt_financing': 1, 'MiddleLow_debt_financing': 2, 'Middle_debt_financing': 3, 'No_debt_financing': 4}, 'Other_Investment_degree': {'High_Other_Investment': 0, 'Low_Other_Investment': 1, 'MiddleLow_Other_Investment': 2, 'Middle_Other_Investment': 3, 'No_Other_Investment': 4}}


In [21]:
df.head()

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,1750000.0,acquired,USA,NY,New York City,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,4000000.0,operating,USA,CA,Los Angeles,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,40000.0,operating,EST,NaN,Tallinn,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,1500000.0,operating,GBR,NaN,London,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,60000.0,operating,USA,TX,Dallas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


> **Round_A_H**

In [22]:
# round_A degree
df1.loc[(df1["round_A"]>= 10000000),"round_A_degree"] = "High_round_A"
df1.loc[(df1["round_A"]< 10000000)& (df1["round_A"]>= 5000000),"round_A_degree"] = "Middle_round_A"
df1.loc[(df1["round_A"]< 5000000)& (df1["round_A"]>= 2000000),"round_A_degree"] = "MiddleLow_round_A"
df1.loc[(df1["round_A"]< 2000000)& (df1["round_A"]>0),"round_A_degree"] = "Low_round_A"
df1.loc[(df1["round_A"]==0),"round_A_degree"] = "No_round_A"
df1['round_A_degree'].value_counts()

No_round_A           35676
MiddleLow_round_A     2664
Middle_round_A        2308
Low_round_A           1876
High_round_A          1619
Name: round_A_degree, dtype: int64

In [23]:
# round_B degree
df1.loc[(df1["round_B"]>= 10000000),"round_B_degree"] = "High_round_B"
df1.loc[(df1["round_B"]< 10000000)& (df1["round_B"]>= 5000000),"round_B_degree"] = "Middle_round_B"
df1.loc[(df1["round_B"]< 5000000)& (df1["round_B"]>= 2000000),"round_B_degree"] = "MiddleLow_round_B"
df1.loc[(df1["round_B"]< 2000000)& (df1["round_B"]>0),"round_B_degree"] = "Low_round_B"
df1.loc[(df1["round_B"]==0),"round_B_degree"] = "No_round_B"
df1['round_B_degree'].value_counts()

No_round_B           38912
High_round_B          2651
Middle_round_B        1426
MiddleLow_round_B      788
Low_round_B            366
Name: round_B_degree, dtype: int64

In [24]:
# round_C degree
df1.loc[(df1["round_C"]>= 10000000),"round_C_degree"] = "High_round_C"
df1.loc[(df1["round_C"]< 10000000)& (df1["round_C"]>= 5000000),"round_C_degree"] = "Middle_round_C"
df1.loc[(df1["round_C"]< 5000000)& (df1["round_C"]>= 2000000),"round_C_degree"] = "MiddleLow_round_C"
df1.loc[(df1["round_C"]< 2000000)& (df1["round_C"]>0),"round_C_degree"] = "Low_round_C"
df1.loc[(df1["round_C"]==0),"round_C_degree"] = "No_round_C"
df1['round_C_degree'].value_counts()

No_round_C           41393
High_round_C          1902
Middle_round_C         504
MiddleLow_round_C      234
Low_round_C            110
Name: round_C_degree, dtype: int64

In [25]:
df1['round_D_H'] = df1['round_D'] +df1['round_E'] +df1['round_F'] +df1['round_G'] +df1['round_H'] 

In [26]:
# round_D_H degree

df1.loc[(df1["round_D_H"]>= 50000000),"round_D_H_degree"] = "High_round_D_H"
df1.loc[(df1["round_D_H"]< 50000000)& (df1["round_D_H"]>= 10000000),"round_D_H_degree"] = "Middle_round_D_H"
df1.loc[(df1["round_D_H"]< 10000000)& (df1["round_D_H"]>= 5000000),"round_D_H_degree"] = "MiddleLow_round_D_H"
df1.loc[(df1["round_D_H"]< 5000000)& (df1["round_D_H"]>0),"round_D_H_degree"] = "Low_round_D_H"
df1.loc[(df1["round_D_H"]==0),"round_D_H_degree"] = "No_round_D_H"
df1['round_D_H_degree'].value_counts()

No_round_D_H           42642
Middle_round_D_H         804
High_round_D_H           353
MiddleLow_round_D_H      196
Low_round_D_H            148
Name: round_D_H_degree, dtype: int64

In [27]:
# 需要编码的列
columns_to_encode = ['round_A_degree', 'round_B_degree', 'round_C_degree', 'round_D_H_degree']

# 创建 LabelEncoder，并创建每个类别到整数的映射
category_mapping = {}

for column in columns_to_encode:
    label_encoder = LabelEncoder()
    df1[column + '_encoded'] = label_encoder.fit_transform(df1[column])
    category_mapping[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# 打印每个类别到整数的映射
print(category_mapping)

{'round_A_degree': {'High_round_A': 0, 'Low_round_A': 1, 'MiddleLow_round_A': 2, 'Middle_round_A': 3, 'No_round_A': 4}, 'round_B_degree': {'High_round_B': 0, 'Low_round_B': 1, 'MiddleLow_round_B': 2, 'Middle_round_B': 3, 'No_round_B': 4}, 'round_C_degree': {'High_round_C': 0, 'Low_round_C': 1, 'MiddleLow_round_C': 2, 'Middle_round_C': 3, 'No_round_C': 4}, 'round_D_H_degree': {'High_round_D_H': 0, 'Low_round_D_H': 1, 'MiddleLow_round_D_H': 2, 'Middle_round_D_H': 3, 'No_round_D_H': 4}}


In [28]:
df1.head()

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region_x,...,Other_Investment_degree_encoded,round_A_degree,round_B_degree,round_C_degree,round_D_H,round_D_H_degree,round_A_degree_encoded,round_B_degree_encoded,round_C_degree_encoded,round_D_H_degree_encoded
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,1750000.0,acquired,USA,NY,New York City,...,4,No_round_A,No_round_B,No_round_C,0.0,No_round_D_H,4,4,4,4
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,4000000.0,operating,USA,CA,Los Angeles,...,4,No_round_A,No_round_B,No_round_C,0.0,No_round_D_H,4,4,4,4
2,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,60000.0,operating,USA,TX,Dallas,...,1,No_round_A,No_round_B,No_round_C,0.0,No_round_D_H,4,4,4,4
3,/organization/club-domains,.Club Domains,http://nic.club/,|Software|,Software,7000000.0,NaN,USA,FL,Ft. Lauderdale,...,4,No_round_A,Middle_round_B,No_round_C,0.0,No_round_D_H,4,3,4,4
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,|Software|,Software,0.0,operating,USA,IL,"Springfield, Illinois",...,4,No_round_A,No_round_B,No_round_C,0.0,No_round_D_H,4,4,4,4


> **Time features**

In [29]:
# 计算创立了多少个月
df1['months_since_founding'] = ((pd.to_datetime('now') - df1['founded_month']).dt.total_seconds() / (30 * 24 * 60 * 60)).fillna(-1).astype(int)

# 计算首次募资花了多少个月
df1['months_to_first_funding'] = ((df1['first_funding_at'] - df1['founded_month']).dt.total_seconds() / (30 * 24 * 60 * 60)).fillna(-1).astype(int)

# 打印结果
df1.head()

C:\Users\12048\AppData\Local\Temp\ipykernel_26496\72585490.py:2: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df1['months_since_founding'] = ((pd.to_datetime('now') - df1['founded_month']).dt.total_seconds() / (30 * 24 * 60 * 60)).fillna(-1).astype(int)


,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region_x,...,round_B_degree,round_C_degree,round_D_H,round_D_H_degree,round_A_degree_encoded,round_B_degree_encoded,round_C_degree_encoded,round_D_H_degree_encoded,months_since_founding,months_to_first_funding
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,1750000.0,acquired,USA,NY,New York City,...,No_round_B,No_round_C,0.0,No_round_D_H,4,4,4,4,140,0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,4000000.0,operating,USA,CA,Los Angeles,...,No_round_B,No_round_C,0.0,No_round_D_H,4,4,4,4,-1,-1
2,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,60000.0,operating,USA,TX,Dallas,...,No_round_B,No_round_C,0.0,No_round_D_H,4,4,4,4,120,7
3,/organization/club-domains,.Club Domains,http://nic.club/,|Software|,Software,7000000.0,NaN,USA,FL,Ft. Lauderdale,...,Middle_round_B,No_round_C,0.0,No_round_D_H,4,3,4,4,148,20
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,|Software|,Software,0.0,operating,USA,IL,"Springfield, Illinois",...,No_round_B,No_round_C,0.0,No_round_D_H,4,4,4,4,169,55


# Round_A Predict

In [30]:
# 选择有用的列
df_A = df1[['sub_region_cat', 'cat_Industry_Group','months_since_founding','months_to_first_funding','seed_degree_encoded', 'angel_degree_encoded', 'debt_financing_degree_encoded', 'Other_Investment_degree_encoded','round_A_degree_encoded']]

In [31]:
df_A.tail(10)

,sub_region_cat,cat_Industry_Group,months_since_founding,months_to_first_funding,seed_degree_encoded,angel_degree_encoded,debt_financing_degree_encoded,Other_Investment_degree_encoded,round_A_degree_encoded
44133,12,8,-1,-1,2,4,4,4,4
44134,11,42,146,6,3,4,4,4,4
44135,11,39,-1,-1,2,4,4,4,4
44136,11,24,160,0,0,4,4,4,4
44137,10,19,129,0,1,4,4,4,4
44138,10,42,204,12,4,4,4,4,4
44139,10,8,147,27,3,4,4,4,4
44140,10,38,181,15,4,4,4,4,4
44141,11,38,147,0,1,4,4,4,4
44142,4,38,279,49,4,4,4,4,4


In [32]:
#筛选有A的进行预测
filtered_df_A=df_A[df_A['round_A_degree_encoded']!=4]

In [33]:
filtered_df_A.head()

,sub_region_cat,cat_Industry_Group,months_since_founding,months_to_first_funding,seed_degree_encoded,angel_degree_encoded,debt_financing_degree_encoded,Other_Investment_degree_encoded,round_A_degree_encoded
11,6,24,163,-6,1,4,4,4,2
25,6,25,206,15,4,4,4,4,0
26,6,1,169,-6,4,4,4,4,2
31,6,38,413,149,4,4,4,4,0
46,6,38,146,19,4,4,4,4,1


In [34]:
from sklearn.model_selection import train_test_split
X = filtered_df_A.drop('round_A_degree_encoded', axis=1)  # 除了 'round_A_degree_encoded' 之外的所有列作为特征
y = filtered_df_A['round_A_degree_encoded']  # 目标变量

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# 定义要使用的模型列表
models = [
    ('KNN', KNeighborsClassifier()),
    ('CART', DecisionTreeClassifier()),
    ('RF', RandomForestClassifier()),
    ('SVC', SVC()),
    ('GBM', GradientBoostingClassifier()),
    ("XGBoost", XGBClassifier(objective='multi:softmax'))
]

In [36]:
best_model_name = None
best_model = None
best_score = 0

# 训练并评估每个模型
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = model.score(X_test, y_test)
    
    if score > best_score:
        best_score = score
        best_model_name = name
        best_model = model
    
    
    print(f'{name} - Accuracy: {score}')
    report = classification_report(y_test, y_pred)
    print(f'{name} - Classification Report:\n{report}')
    
print(f'Best Model: {best_model_name} - Best Accuracy: {best_score}')

KNN - Accuracy: 0.29811097992916175
KNN - Classification Report:
              precision    recall  f1-score   support

           0       0.27      0.30      0.28       334
           1       0.27      0.28      0.27       379
           2       0.34      0.40      0.37       516
           3       0.29      0.20      0.24       465

    accuracy                           0.30      1694
   macro avg       0.29      0.29      0.29      1694
weighted avg       0.30      0.30      0.29      1694

CART - Accuracy: 0.30460448642266824
CART - Classification Report:
              precision    recall  f1-score   support

           0       0.28      0.28      0.28       334
           1       0.27      0.27      0.27       379
           2       0.33      0.35      0.34       516
           3       0.32      0.30      0.31       465

    accuracy                           0.30      1694
   macro avg       0.30      0.30      0.30      1694
weighted avg       0.30      0.30      0.30      1694

D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GBM - Accuracy: 0.36363636363636365
GBM - Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.26      0.30       334
           1       0.42      0.24      0.30       379
           2       0.35      0.64      0.45       516
           3       0.35      0.24      0.29       465

    accuracy                           0.36      1694
   macro avg       0.37      0.34      0.34      1694
weighted avg       0.37      0.36      0.34      1694

XGBoost - Accuracy: 0.34061393152302244
XGBoost - Classification Report:
              precision    recall  f1-score   support

           0       0.34      0.27      0.30       334
           1       0.38      0.27      0.32       379
           2       0.34      0.49      0.40       516
           3       0.32      0.28      0.29       465

    accuracy                           0.34      1694
   macro avg       0.35      0.33      0.33      1694
weighted avg       0.34      0.34      0.33    

In [37]:
feature_importance = best_model.feature_importances_
feature_names = X_train.columns 

# 创建一个 DataFrame 来存储特征重要性
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

# 按照重要性降序排列
sorted_feature_importance = feature_importance_df.sort_values(by='Importance', ascending=False)

# 输出排好序的特征重要性
print(sorted_feature_importance)


                           Feature  Importance
3          months_to_first_funding    0.202887
1               cat_Industry_Group    0.199869
2            months_since_founding    0.191631
0                   sub_region_cat    0.174451
4              seed_degree_encoded    0.124264
5             angel_degree_encoded    0.042379
6    debt_financing_degree_encoded    0.041003
7  Other_Investment_degree_encoded    0.023517


## Hyperparameter Tuning

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# 定义要搜索的超参数范围
param_grid = {
    'learning_rate': [0.1, 0.05, 0.01],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5]
}

# 创建 GradientBoostingClassifier 模型
base_model = GradientBoostingClassifier()

# 创建 GridSearchCV 实例，并设置 refit=True
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1, refit=True)

# 在训练数据上进行搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best parameters found: ", grid_search.best_params_)

# 输出在整个训练集上的最佳模型的准确率
y_pred_train = grid_search.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print("Accuracy on train set with best model: {:.2f}%".format(accuracy_train * 100))

# 输出在测试集上的准确率
y_pred_test = grid_search.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
print("Accuracy on test set with best model: {:.2f}%".format(accuracy_test * 100))


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best parameters found:  {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
Accuracy on train set with best model: 41.22%
Accuracy on test set with best model: 36.78%


## PCA

在这里，我们尝试用PCA对数据降维，但降维后的模型效果不如降维前，因此放弃使用PCA方法。

In [38]:
from sklearn.decomposition import PCA

# 创建 PCA 模型并拟合数据
pca = PCA(n_components=4)  # 选择要降到的维度
X_pca = pca.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [39]:
best_model_name = None
best_model = None
best_score = 0

# 训练并评估每个模型
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = model.score(X_test, y_test)
    
    if score > best_score:
        best_score = score
        best_model_name = name
        best_model = model
    
    
    print(f'{name} - Accuracy: {score}')
    report = classification_report(y_test, y_pred)
    print(f'{name} - Classification Report:\n{report}')
    
print(f'Best Model: {best_model_name} - Best Accuracy: {best_score}')

KNN - Accuracy: 0.2733175914994097
KNN - Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.30      0.27       334
           1       0.24      0.25      0.25       379
           2       0.31      0.34      0.33       516
           3       0.27      0.20      0.23       465

    accuracy                           0.27      1694
   macro avg       0.27      0.27      0.27      1694
weighted avg       0.27      0.27      0.27      1694

CART - Accuracy: 0.29161747343565525
CART - Classification Report:
              precision    recall  f1-score   support

           0       0.26      0.27      0.26       334
           1       0.27      0.28      0.27       379
           2       0.32      0.33      0.32       516
           3       0.31      0.27      0.29       465

    accuracy                           0.29      1694
   macro avg       0.29      0.29      0.29      1694
weighted avg       0.29      0.29      0.29      1694


# Round_B Predict

In [36]:
df_B = df1[['sub_region_cat', 'cat_Industry_Group','months_since_founding','months_to_first_funding','seed_degree_encoded','angel_degree_encoded', 'debt_financing_degree_encoded', 'Other_Investment_degree_encoded','round_A_degree_encoded','round_B_degree_encoded']]


In [37]:
#筛选有B的进行预测
filtered_df_B=df_B[df_B['round_B_degree_encoded']!=4]

In [38]:
filtered_df_B.tail()

,sub_region_cat,cat_Industry_Group,months_since_founding,months_to_first_funding,seed_degree_encoded,angel_degree_encoded,debt_financing_degree_encoded,Other_Investment_degree_encoded,round_A_degree_encoded,round_B_degree_encoded
44027,12,23,206,58,4,4,4,4,4,3
44028,12,17,206,0,4,4,4,4,1,3
44062,4,23,206,22,4,4,4,4,3,3
44104,9,1,193,32,3,4,3,4,2,0
44106,9,7,141,1,1,1,4,4,1,1


In [39]:
#predict

In [55]:
X = filtered_df_B.drop('round_B_degree_encoded', axis=1)  # 除了 'round_A_degree_encoded' 之外的所有列作为特征
y = filtered_df_B['round_B_degree_encoded']  # 目标变量

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
best_model_name = None
best_model = None
best_score = 0

# 训练并评估每个模型
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = model.score(X_test, y_test)
    
    if score > best_score:
        best_score = score
        best_model_name = name
        best_model = model
    
    
    print(f'{name} - Accuracy: {score}')
    report = classification_report(y_test, y_pred)
    print(f'{name} - Classification Report:\n{report}')
    
print(f'Best Model: {best_model_name} - Best Accuracy: {best_score}')

KNN - Accuracy: 0.44126074498567336
KNN - Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.76      0.60       506
           1       0.10      0.04      0.06        77
           2       0.16      0.08      0.10       146
           3       0.36      0.20      0.26       318

    accuracy                           0.44      1047
   macro avg       0.28      0.27      0.26      1047
weighted avg       0.38      0.44      0.39      1047

CART - Accuracy: 0.4221585482330468
CART - Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.59      0.57       506
           1       0.13      0.10      0.12        77
           2       0.21      0.23      0.22       146
           3       0.36      0.31      0.34       318

    accuracy                           0.42      1047
   macro avg       0.31      0.31      0.31      1047
weighted avg       0.41      0.42      0.42      1047


D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GBM - Accuracy: 0.4899713467048711
GBM - Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.84      0.65       506
           1       0.19      0.08      0.11        77
           2       0.29      0.08      0.13       146
           3       0.40      0.22      0.29       318

    accuracy                           0.49      1047
   macro avg       0.35      0.31      0.29      1047
weighted avg       0.43      0.49      0.43      1047

XGBoost - Accuracy: 0.4890162368672397
XGBoost - Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.75      0.64       506
           1       0.31      0.14      0.20        77
           2       0.25      0.14      0.18       146
           3       0.40      0.31      0.35       318

    accuracy                           0.49      1047
   macro avg       0.38      0.34      0.34      1047
weighted avg       0.45      0.49      0.46      

In [46]:
feature_importance = best_model.feature_importances_
feature_names = X_train.columns  # 假设 X_train 是你用于训练的特征集合

# 创建一个 DataFrame 来存储特征重要性
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

# 按照重要性降序排列
sorted_feature_importance = feature_importance_df.sort_values(by='Importance', ascending=False)

# 输出排好序的特征重要性
print(sorted_feature_importance)


                           Feature  Importance
8           round_A_degree_encoded    0.411991
2            months_since_founding    0.164762
3          months_to_first_funding    0.145640
1               cat_Industry_Group    0.117657
0                   sub_region_cat    0.083744
6    debt_financing_degree_encoded    0.032633
4              seed_degree_encoded    0.018626
7  Other_Investment_degree_encoded    0.015859
5             angel_degree_encoded    0.009088


## Hyperparameter Tuning

In [56]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# 定义要搜索的超参数范围
param_grid = {
    'learning_rate': [0.1, 0.05, 0.01],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5]
}

# 创建 GradientBoostingClassifier 模型
base_model = GradientBoostingClassifier()

# 创建 GridSearchCV 实例，并设置 refit=True
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1, refit=True)

# 在训练数据上进行搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best parameters found: ", grid_search.best_params_)

# 输出在整个训练集上的最佳模型的准确率
y_pred_train = grid_search.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print("Accuracy on train set with best model: {:.2f}%".format(accuracy_train * 100))

# 输出在测试集上的准确率
y_pred_test = grid_search.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
print("Accuracy on test set with best model: {:.2f}%".format(accuracy_test * 100))


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best parameters found:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Accuracy on train set with best model: 59.80%
Accuracy on test set with best model: 49.09%


# Round C predict

In [36]:
df_C = df1[['sub_region_cat', 'cat_Industry_Group','months_since_founding','months_to_first_funding','seed_degree_encoded', 'angel_degree_encoded', 'debt_financing_degree_encoded', 'Other_Investment_degree_encoded','round_A_degree_encoded','round_B_degree_encoded','round_C_degree_encoded']]


In [37]:
#筛选有C的进行预测
filtered_df_C=df_C[df_C['round_C_degree_encoded']!=4]

In [38]:
filtered_df_C.tail()

,sub_region_cat,cat_Industry_Group,months_since_founding,months_to_first_funding,seed_degree_encoded,angel_degree_encoded,debt_financing_degree_encoded,Other_Investment_degree_encoded,round_A_degree_encoded,round_B_degree_encoded,round_C_degree_encoded
43958,8,7,144,9,4,4,4,4,0,0,0
44028,12,17,206,0,4,4,4,4,1,3,2
44062,4,23,206,22,4,4,4,4,3,3,0
44104,9,1,193,32,3,4,3,4,2,0,0
44142,4,38,279,49,4,4,4,4,4,4,0


In [39]:
# predict

In [40]:
X = filtered_df_C.drop('round_C_degree_encoded', axis=1)  # 除了 'round_A_degree_encoded' 之外的所有列作为特征
y = filtered_df_C['round_C_degree_encoded']  # 目标变量

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
best_model_name = None
best_model = None
best_score = 0

# 训练并评估每个模型
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = model.score(X_test, y_test)
    
    if score > best_score:
        best_score = score
        best_model_name = name
        best_model = model
    
    
    print(f'{name} - Accuracy: {score}')
    report = classification_report(y_test, y_pred)
    print(f'{name} - Classification Report:\n{report}')
    
print(f'Best Model: {best_model_name} - Best Accuracy: {best_score}')

KNN - Accuracy: 0.6672727272727272
KNN - Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.91      0.80       394
           1       0.00      0.00      0.00        22
           2       0.29      0.11      0.16        46
           3       0.12      0.06      0.08        88

    accuracy                           0.67       550
   macro avg       0.29      0.27      0.26       550
weighted avg       0.56      0.67      0.60       550

CART - Accuracy: 0.5818181818181818
CART - Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.75      0.75       394
           1       0.08      0.09      0.08        22
           2       0.12      0.13      0.12        46
           3       0.21      0.20      0.21        88

    accuracy                           0.58       550
   macro avg       0.29      0.29      0.29       550
weighted avg       0.59      0.58      0.59       550



D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RF - Accuracy: 0.6927272727272727
RF - Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.92      0.83       394
           1       0.30      0.14      0.19        22
           2       0.07      0.02      0.03        46
           3       0.31      0.17      0.22        88

    accuracy                           0.69       550
   macro avg       0.36      0.31      0.32       550
weighted avg       0.61      0.69      0.64       550

SVC - Accuracy: 0.7163636363636363
SVC - Classification Report:
              precision    recall  f1-score   support

           0       0.72      1.00      0.83       394
           1       0.00      0.00      0.00        22
           2       0.00      0.00      0.00        46
           3       0.00      0.00      0.00        88

    accuracy                           0.72       550
   macro avg       0.18      0.25      0.21       550
weighted avg       0.51      0.72      0.60       550



D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GBM - Accuracy: 0.7
GBM - Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.94      0.84       394
           1       0.33      0.09      0.14        22
           2       0.00      0.00      0.00        46
           3       0.26      0.12      0.17        88

    accuracy                           0.70       550
   macro avg       0.33      0.29      0.29       550
weighted avg       0.59      0.70      0.63       550

XGBoost - Accuracy: 0.6654545454545454
XGBoost - Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.88      0.82       394
           1       0.14      0.09      0.11        22
           2       0.18      0.09      0.12        46
           3       0.23      0.17      0.20        88

    accuracy                           0.67       550
   macro avg       0.33      0.31      0.31       550
weighted avg       0.61      0.67      0.63       550

Best Mode

## Hyperparameter Tuning

In [44]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 定义要搜索的较小超参数范围
param_grid = {
    'C': [0.1, 1],
    'gamma': [0.1, 0.01],
    'kernel': ['rbf', 'linear']
}

# 创建 SVM 模型
base_model = SVC()

# 创建 GridSearchCV 实例
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1, refit=True)

# 在训练数据上进行搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数和其对应的准确率
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# 使用最佳模型预测测试集
y_pred_test = best_model.predict(X_test)

# 计算预测准确率
accuracy = accuracy_score(y_test, y_pred_test)

print("Best parameters found: ", best_params)
print("Accuracy with best parameters on test set: {:.2f}%".format(accuracy * 100))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters found:  {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy with best parameters on test set: 71.64%


# Round D_H predict

In [47]:
df_D_H = df1[['sub_region_cat', 'cat_Industry_Group','months_since_founding','months_to_first_funding','seed_degree_encoded', 'angel_degree_encoded', 'debt_financing_degree_encoded', 'Other_Investment_degree_encoded','round_A_degree_encoded','round_B_degree_encoded','round_C_degree_encoded','round_D_H_degree_encoded']]


In [48]:
#筛选有D_H的进行预测
filtered_df_D_H=df_D_H[df_D_H['round_D_H_degree_encoded']!=4]

In [49]:
filtered_df_D_H.tail()

,sub_region_cat,cat_Industry_Group,months_since_founding,months_to_first_funding,seed_degree_encoded,angel_degree_encoded,debt_financing_degree_encoded,Other_Investment_degree_encoded,round_A_degree_encoded,round_B_degree_encoded,round_C_degree_encoded,round_D_H_degree_encoded
43715,8,7,181,14,4,4,4,4,4,4,4,0
43947,8,33,315,59,4,4,4,4,4,4,3,3
43951,8,8,279,23,4,4,4,4,2,0,0,3
43958,8,7,144,9,4,4,4,4,0,0,0,0
44142,4,38,279,49,4,4,4,4,4,4,0,3


In [50]:
# predict

In [51]:
X = filtered_df_D_H.drop('round_D_H_degree_encoded', axis=1)  # 除了 'round_A_degree_encoded' 之外的所有列作为特征
y = filtered_df_D_H['round_D_H_degree_encoded']  # 目标变量

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
best_model_name = None
best_model = None
best_score = 0

# 训练并评估每个模型
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = model.score(X_test, y_test)
    
    if score > best_score:
        best_score = score
        best_model_name = name
        best_model = model
    
    
    print(f'{name} - Accuracy: {score}')
    report = classification_report(y_test, y_pred)
    print(f'{name} - Classification Report:\n{report}')
    
print(f'Best Model: {best_model_name} - Best Accuracy: {best_score}')

KNN - Accuracy: 0.48172757475083056
KNN - Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.34      0.33        62
           1       0.19      0.08      0.12        36
           2       0.18      0.09      0.12        34
           3       0.58      0.70      0.63       169

    accuracy                           0.48       301
   macro avg       0.32      0.30      0.30       301
weighted avg       0.43      0.48      0.45       301

CART - Accuracy: 0.4485049833887043
CART - Classification Report:
              precision    recall  f1-score   support

           0       0.32      0.34      0.33        62
           1       0.15      0.11      0.13        36
           2       0.13      0.15      0.14        34
           3       0.62      0.62      0.62       169

    accuracy                           0.45       301
   macro avg       0.30      0.30      0.30       301
weighted avg       0.44      0.45      0.45       301


D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GBM - Accuracy: 0.5581395348837209
GBM - Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.32      0.37        62
           1       0.50      0.22      0.31        36
           2       0.12      0.03      0.05        34
           3       0.60      0.82      0.69       169

    accuracy                           0.56       301
   macro avg       0.42      0.35      0.36       301
weighted avg       0.50      0.56      0.51       301

XGBoost - Accuracy: 0.5083056478405316
XGBoost - Classification Report:
              precision    recall  f1-score   support

           0       0.34      0.27      0.30        62
           1       0.42      0.14      0.21        36
           2       0.14      0.09      0.11        34
           3       0.59      0.76      0.66       169

    accuracy                           0.51       301
   macro avg       0.37      0.31      0.32       301
weighted avg       0.47      0.51      0.47      

## Hyperparameter Tuning

In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 定义要搜索的较小超参数范围
param_grid = {
    'C': [0.1, 1],
    'gamma': [0.1, 0.01],
    'kernel': ['rbf', 'linear']
}

# 创建 SVM 模型
base_model = SVC()

# 创建 GridSearchCV 实例
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1, refit=True)

# 在训练数据上进行搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数和其对应的准确率
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# 使用最佳模型预测测试集
y_pred_test = best_model.predict(X_test)

# 计算预测准确率
accuracy = accuracy_score(y_test, y_pred_test)

print("Best parameters found: ", best_params)
print("Accuracy with best parameters on test set: {:.2f}%".format(accuracy * 100))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters found:  {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Accuracy with best parameters on test set: 56.15%


In [54]:
# 获取最佳模型
best_model = grid_search.best_estimator_

# 获取特征系数
feature_coef = best_model.coef_[0]

# 获取特征名称
feature_names = X_train.columns.tolist()

# 创建 DataFrame 来存储特征系数
feature_coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': feature_coef})

# 按照系数绝对值降序排列
sorted_feature_coef = feature_coef_df.reindex(feature_coef_df['Coefficient'].abs().sort_values(ascending=False).index)

# 输出排好序的特征系数
print(sorted_feature_coef)


                            Feature  Coefficient
7   Other_Investment_degree_encoded    -0.262383
10           round_C_degree_encoded    -0.210118
8            round_A_degree_encoded    -0.138207
5              angel_degree_encoded    -0.106913
9            round_B_degree_encoded    -0.086435
4               seed_degree_encoded    -0.050793
6     debt_financing_degree_encoded    -0.027633
1                cat_Industry_Group    -0.022866
0                    sub_region_cat     0.013442
3           months_to_first_funding    -0.006533
2             months_since_founding     0.005964
